# SIXT33N Project Phase 3: Controls
## EECS 16B: Designing Information Devices and Systems II, Fall 2021

## Table of Contents

* [Introduction](#intro)
 - [Lab Note](https://drive.google.com/file/d/1WWjwVdGvuB0IjyAI769HBGd1GrCX2tvO/view?usp=sharing)
* [Part 1: Open Loop Control](#part1)
* [Part 2.1: Closed Loop Design](#part21)
* [Part 2.2: Closed Loop Simulation and f-Value Selection](#part22)
* [Part 2.3: Closed Loop Implementation](#part23)
* [Part 2.4: f-Value Tuning](#part24)
* [Part 2.5: Steady-State Error Correction](#part25)
* [Part 3: Open Loop Comparison](#part3)
* [Part 4: Micboard Verification](#part4)


<a id='part0'></a>
# <span style="color:blue">(Review) Part 0: Powering the MSP</span>

Powering the MSP by both the 5V from the voltage regulator and the USB can increase the odds of MSP frying. To avoid this, you should have connected the 5V ouput of the voltage regulator to the 5V pin of the MSP via a 20 ohm resistor (two 10 ohm resistors in series). This schematic is illustrated below: 
<img style="width:700px" src="images/20ohm_regulator.png">
Despite this modification, always avoid simulataneously powering the MSP by the above circuit and the USB. Only use one source at a time. 

-----

<a id='part1'></a>
# <span style="color:blue">Part 1: Open Loop Control</span>

## 1.0 Introduction

### [Lab Note](https://drive.google.com/file/d/1WWjwVdGvuB0IjyAI769HBGd1GrCX2tvO/view?usp=sharing)

Read the part 1 of the note to familiarize yourself with/remind yourself of the open-loop model.
Last time, in System ID, you modeled the open-loop system, collected data, and determined the unknown parameters of your car.

This week, you'll implement the open-loop and closed-loop control systems. By the end of this lab, you should be able to control the trajectory of the car to drive straight. We will address turning later in the project. **Remember to document all design choices you made and explain them in the project report.**

The goals of this phase are as follows:
- Controller design
- Closed loop simulation
- Drive at straight at a constant speed

### <span style="color:red">**Fill in your model parameters and target velocity from System ID below.**</span>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import utils
%matplotlib inline

In [ ]:
# YOUR PARAMETERS HERE
theta_left = ...;
theta_right = ...;
beta_left = ...;
beta_right = ...;
v_star = ...;

simulator = utils.Simulator(theta_left, theta_right, beta_left, beta_right)

In [ ]:
params = np.array([(theta_left, theta_right),
                   (beta_left, beta_right)])
d0 = (0,  0)
sim_length = 10 # sim length
mismatch_error = 0.1 # 10% model mismatch

<a id='part1.1'></a>
## 1.1 Open Loop Controller Design

### <a href="https://drive.google.com/file/d/1WWjwVdGvuB0IjyAI769HBGd1GrCX2tvO/view?usp=sharing">Read Part 1 of the note before you proceed. Boxed questions make good checkoff questions ;)</a>

Now design an open loop controller that uses the desired wheel velocity, `v_star`, to set the input `u` to an appropriate value for the pulse width modulation (PWM). To do this, solve your model from the previous part for the input $u[n]$ that makes the model velocity $d[n+1] - d[n]$ equal to the desired velocity $v^*$.

In [ ]:
# DESIGN YOUR OPEN-LOOP CONTROL HERE
def drive_straight_left_ol(v_star):
    """Returns the open loop input 'u' for the left wheel"""
    # YOUR CODE HERE
    return ...

def drive_straight_right_ol(v_star):
    """Returns the open loop input 'u' for the right wheel"""
    # YOUR CODE HERE
    return ...

<a id='part1.2'></a>
## 1.2 Open Loop Simulation


Now, let's do a quick simulation to check the behavior of the car based on our functions above. In these simulations, we apply a maximum PWM for a short time as we do in the real car to "jolt" the motors from rest.

**Important:** As long as $\delta$ is constant, even if $\delta\neq 0$, the car still travels straight! Traveling straight means the *velocities*, not the positions, of the wheels are the same. It may just travel straight in a different direction than the original heading. Lets call the steady state (final) value $\delta[n]\overset{n\rightarrow \infty}{=}\delta_{ss}$.


## Mismatch
In the System ID lab, we found $\beta_{L,R}$ and $\theta_{L,R}$ to model two wheels. However, there could be a mismatch between the estimated model parameters $\beta_{L,R}, \theta_{L,R}$ and real model parameters $\beta^*_{L,R}, theta^*_{L,R}.$ In this simulation, we assumed that the mismatch error is the same for both $\beta_{L,R} and \theta_{L,R}$


$$\theta^*_{L,R}=\theta_{L,R} \times (1+\text{mismatch_error})$$
$$\beta^*_{L,R}=\beta_{L,R} \times (1+\text{mismatch_error})$$


In the left two plots, we assume the physical system perfectly matches our model. On the right, we assume the physical system is 10% different from our model, meaning the model parameters of the wheels have a 10% mismatch.

The 5th plot titled "Trajectory of the car" shows how the car would travel depending on which model is used. In the perfect model with the constant delta, the car is driving straight. In the case with model mismatch, and a growing delta, the car continues to spin right.

**Sanity check: Why is the car spinning right and not left? Hint: what does delta represent?"**


In [ ]:
titles = ["Open-loop control with perfect model",
          "Open-loop control with model mismatch"]
_ = utils.two_sims(titles, simulator, v_star, drive_straight_left_ol, drive_straight_right_ol)
utils.plot_car_traj(titles[1], simulator, v_star, drive_straight_left_ol, drive_straight_right_ol,mismatch_error=0.1,sim_length=sim_length)

From the above plots, we can see that the open-loop car travels straight only with a perfect model.

<a id='part1.3'></a>
## 1.3 Jolt Calculation

### <a href="https://drive.google.com/file/d/1WWjwVdGvuB0IjyAI769HBGd1GrCX2tvO/view?usp=sharing">Before you proceed, read the "Jolt Calculation" section of Part 1 of the note.</a>

We find the jolts below by calculating the inputs that result in the minimum of the maximum velocities of both wheels. 

- **Copy your `data_coarse.txt` file from System ID into the same directory as this iPython notebook.**
- Run the code below to find your initial jolt values.
- Your initial jolt values are correct if they are the input values that correspond to the points where the blue and yellow lines intersect the green line respectively.

In [ ]:
left_jolt, right_jolt = utils.find_jolt()
print('int left_jolt = {};\nint right_jolt = {};'.format(left_jolt, right_jolt))

<a id='part1.4'></a>
## 1.4 Open Loop Implementation

Now we will implement the controller on the Launchpad. The sketch **`open_loop.ino`** contains some code that implements the open loop control.

- **If you've changed any of the pins from their default values in System ID, make sure those changes are reflected in the pin definitions.**

- Fill in these values in `CODE BLOCK CON1`:
    - desired velocity: `v_star` 
    - estimated parameters: `theta_left`, `theta_right`, `beta_left`, `beta_right`
    - initial jolt values: `left_jolt`, `right_jolt`
    
    
- Additionally, we need to compute the PWM input to the motors in `CODE BLOCK CON2`:
    - Complete `driveStraight_left()` and `driveStraight_right()` to compute `left_cur_pwm` and `right_cur_pwm` (the PWM inputs for each wheel required for a desired v_star)
    - **Note: Energia code requires using C/C++ syntax, rather than the Python syntax you used to write the drive_straight_left_ol and drive_straight_right_ol functions above. Make sure you only copy the body of the function over into open_loop.ino, not the def statement itself. Depending on how you wrote the function, you may need to modify the syntax to fit C/C++ syntax.**
    - Hint: Your $\theta$ and $\beta$ values are defined globally at the top of the file, so you are able to reference them inside your function.
    
    
- Fill out `CODE BLOCK CON0` to calculate the control inputs using the functions from `CODE BLOCK CON2`.
- Upload the sketch to your Launchpad. Press the leftmost button (2.1) to start the car to test that your motors are working.
- **Make sure that the car is suspended above the desk.** Your car wheels are going to move in this step.
- <span style="color:red">**Remember, never plug in both the 5V pin on the Launchpad and the USB at the same time!**</span>

Once you are satisfied that your motors work and your open loop control is implemented, put the car on the ground. Make sure the castor wheel is still straight, or else your car may already turn upon starting. Make sure the motor switch is on (and that both your motors are still plugged in) and press the `RST` button, then leftmost (2.1) button on your Launchpad. SIXT33N should move straighter than in System ID because you have accounted for differences in the two motors. Does it behave how you expect? It's okay if it doesn't drive completely straight at this stage, as it most likely will still turn due to model mismatch and disturbances/imperfections in the real world.

-----

<a id='part2.0'></a>
# <span style="color:blue">Part 2: Closed Loop Control</span>

<a id='part21'></a>
## 2.1 Closed Loop Design

Given the data you found in System ID, we know that the car does not match perfectly with our linear model *(think about why we know this)*. In order to correct for the imperfect model, we can build a closed loop controller that uses feedback to ensure that both wheels still drive at the same velocity. The controller has 2 inputs, namely the desired velocity and the difference between the wheel distances $\delta[n] = d_L[n] - d_R[n]$.

### 1. Read part 2 of [the lab note](https://drive.google.com/file/d/1WWjwVdGvuB0IjyAI769HBGd1GrCX2tvO/view?usp=sharing) before you begin.

Now that we've derived the model as shown in the lab note, we can begin implementing it. Copy your $\theta$, $\beta$, and v* values down below, and fill in the two functions.

In [ ]:
# YOUR PARAMETERS HERE
theta_left = ...
theta_right = ...
beta_left = ...
beta_right = ...
v_star = ...

simulator = utils.Simulator(theta_left, theta_right, beta_left, beta_right)

In [ ]:
# DESIGN YOUR CLOSED-LOOP CONTROL HERE USING f_left AND f_right AS YOUR FEEDBACK TERMS
def drive_straight_left_cl(v_star, delta):
    """Returns the closed loop input 'u' for the left wheel"""
    # YOUR CODE HERE
    return ...

def drive_straight_right_cl(v_star, delta):
    """Returns the closed loop input 'u' for the right wheel"""
    # YOUR CODE HERE
    return ...

<a id='part22'></a>
## 2.2 Closed Loop Simulation and f-Value Selection

Next we need to choose values for the control gains so that the $\delta[n]$ update equation is stable. *Concept check: For what eigenvalues is a discrete system stable?* There should be a range of parameter values that make it stable, so we can tune the gains to get the desired performance.

Take a look at each pair of perfect/mismatched-model plots below for each f-value pair and relate how our choice in f-values affects the car's performance. For each pair of plots, ask yourself these questions:
1. Does the car seem to drive straight in the perfectly matched model?
2. Does the car seem to correct for any errors that arise from a mismatched model? Does it end up driving straight?
3. How well does the car correct for errors?
3. Why do the chosen f-values make our system exhibit such behaviors?

<span style='color:red'>**Try the 5 different sets of gains below and see the effect of each.**</span>

In [ ]:
f_left, f_right = None, None
steady_state_error = None

# Maps description of f_values to (f_left, f_right)
f_values = {
    "marginally stable": (-0.3, 0.3),
    "stable, corrects error in one step": (0.5, 0.5),
    "oscillatory, marginally stable": (1, 1),
    "oscillatory, unstable": (1.1, 1.1),
    "stable": (0.2, 0.2),
    "chosen f values": (0, 0)  # TODO: record your chosen f-values here
}

for key in f_values:
    f_left, f_right = f_values[key]

    titles = ["Closed-loop control with perfect model",
              "Closed-loop control with model mismatch, fL={}, fR={}".format(f_left, f_right)]
    _, delta = utils.two_sims(titles, simulator, v_star, drive_straight_left_cl, drive_straight_right_cl)

    print("fL={}, fR={}".format(f_left, f_right))
    print("Eigenvalue of system: 1-fL-fR={:.2g}; {}".format(1-f_left-f_right, key))
    plt.show()
    steady_state_error = delta[-1]

Now that you've taken a look at the behavior of various different values for f_left and f_right, choose your own for your car! Fill in your chosen values in the code above to see how your set compares to the behavior of the other sets of values. Does it do what you expected it to?

<a id='part23'></a>
## 2.3 Closed Loop Implementation

Now implement the controller on the Launchpad. The sketch **`closed_loop.ino`** contains some code that implements the closed loop control. 

- If you've changed any of the pins from their default values, make sure those changes are reflected in the pin definitions.
- Fill out `CODE BLOCK CON1` with your control gains **`f_left`** and **`f_right`** as well as your theta, beta, and v_star values from System ID.
    - A good starting point for controlling your car would be to set `f_left` = `f_right` = 0.5. Finding the values that work best for your car is an iterative process, and you'll have to fine tune these values in order to have the car drive straight.
- Fill out `CODE BLOCK CON2` with your closed-loop control scheme.
    - **As a reminder, Energia code uses C/C++ syntax! You might not be able to simply copy and paste and expect it to compile perfectly.**
- Fill out `CODE BLOCK CON0` in the body of the loop function to calculate the control inputs. Inspect this code to see how the control system is implemented. For now, ignore the term `delta_ss`, as it's currently set to 0.
- Upload the sketch to your LaunchPad.
    - Make sure that the car is suspended above the desk. Your car wheels are going to move in this step.
    - Check that the wheels are moving as you expect them to.
    - **If you use your hand to put some friction one of the wheels (i.e. slow it down), does the feedback correct for it?**

Once you are satisfied that your motors work and your closed-loop control is implemented, run the car on the ground. Make sure the castor wheel is straight, or else your car may already turn upon starting. If it behaves erroneously (for example the controller oscillates), try different eigenvalues. Our simulations compare our possibly mismatched linear model with a linear approximation of our car. In reality, the car is a nonlinear system, so the simulations may not align perfectly with reality.

**Note:** It's possible that the car curves *slightly* over long distances, even if it the car thinks it's traveling straight. This can be due to factors such as mechanical differences past the shaft of the motor, including slight variation in wheel size or wheel wobble. We can fix this later in phase 5 of the project.

<span style='color:red'>**What are some reasons why the simulation might behave differently from the car? What about the differences between when the car is on the ground versus when it is on the desk?**</span>

<a id='part24'></a>
## 2.4 f-Value Tuning

How did the car behave? It probably didn't go perfectly straight with the first set of f values that you picked. In order to find the "right" f values, we will need to analyze the behavior of our car. Recall in lecture that we learned about how the control values will affect the convergence of a system. For our system, we want $\delta[n]$ (the blue line) to converge to 0 in a reasonable amount of time.

small f-values | large f-values
- | - 
![alt](images/small_f.png) | ![alt](images/large_f.png)
 
The above plots can be summarized as:
- if f_left and f_right are too small, it takes a long time to converge to our desired solution.
- if f_left and f_right are too big, oscillations happen.

We can use this information to help guide us to f values that get our car going straight. More example cases are in the lab note, and check out [this youtube playlist](https://www.youtube.com/playlist?list=PLQ1OVCqw6RUPgCmv8H6y9qbcMBT8lotDt) if you need some examples of what each case could look like (note that the playlist call them "k values" instead of "f values").

You will likely have to try several sets of f values until you find one that works well with your particular car. If you find yourself having a lot of trouble narrowing in on some, ask your GSI for help!

Tips: 
 - Don't randomly guess on the f values! This will take you a long time to find the ones that make your car drive straight. Your tuning should always be based off of the previous iterations' behavior; don't just randomly choose values. Try to be smart about how you change the f values from iteration to iteration. If your car is turning to the left, think about how to change each f value, either by increasing or decreasing its value, to fix it.
 - If you're trying to figure out whether to increase or decrease your f_left or f_right based off of your car's behavior, you can try setting the values to two opposite extremes (ie f_left = .9, f_right = .1 or f_left = .1, f_right = .9) and test it to see which one makes your car drive straighter.


<a id='part25'></a>

## 2.5 Steady State Error Correction

Lastly, what if delta doesn't converge to 0, but to some non-zero constant due to model mismatch? This means that the car is driving straight, but not with the same bearing as you placed the car down with (i.e. the car has turned but drives straight eventually). However, we want the car to travel in (approximately) the same direction as when it starts moving. We can accomplish this by manually add the known steady state error $\delta_{ss}$ to each calculation of the error $\delta$. Accounting for the known $\delta_{ss}$ helps the model better respond to unexpected pertubations affecting $\delta$, increasing the model's control strength.

We now calculate $\delta[n]=d_L - d_R + \delta_{ss}$.


In [ ]:
plt.figure(figsize=(5, 7))
plt.subplot(211)
# Simulate using steady_state_error
d = simulator.simulate(
    v_star, drive_straight_left_cl, drive_straight_right_cl, mismatch_error=0.1, sim_length=20, offset=steady_state_error)
delta = simulator.plot(d)
plt.title("Closed loop control with model mismatch and\nsteady state error correction, fL={}, fR={}"
          .format(f_left, f_right))
plt.subplot(212)
plt.plot(delta, 'r')
plt.ylabel('delta')
plt.xlabel('n(sample)')
plt.grid()

- `closed_loop.ino` collects data as it runs, including $\delta[n]$. Like `dynamics_data.ino` from the System ID lab, data will be written to Flash Memory at the end of the run so that data is not lost upon cutting off power to the Launchpad.
    - Set `WRITE = 1` at the top of the `closed_loop.ino` program.
    - Upload the program, and press the leftmost button (2.1) when you are ready to run the car.
    - Let the car run until it settles to (approximately) straight, it should do so by the time it naturally comes to a stop in the code.
    - First unplug the batteries from the car, and then unplug the `5V` pin of your launchpad from your regulator circuits.
    - Set `WRITE = 0` at the top of the `closed_loop.ino` program.
    - Upload the program to your launchpad and open the serial monitor (baud rate 38400). Press the leftmost button (2.1) to start the code and print the collected delta values to the monitor.  
    
- Note the steady-state value $\delta_{ss}$.
    - The first column of the printed data is $\delta$ over time.
    - **If $\delta$ doesn't reach steady-state (converging to a value), the car is not going straight according to the sensors.**
        - It's okay if it isn't exactly constant.
        - Just be sure the variation is small in proportion to the average value.

## Implementing Steady State Error Correction

- Input your value of $\delta_{ss}$ in `CODE BLOCK CON3` in **`closed_loop.ino`**.
- Convince yourself that we need to *add* this value in `CODE BLOCK CON0`.

For example, if the car drives straight but turned right from its initial heading, we want the right wheel to farther so it turns back towards the left. Adding a positive number to delta accomplishes this.

<a id='part3'></a>

# <span style="color:blue">Part 3: Open Loop Comparison</span>

To compare your control scheme to an open-loop model, you can set both of your f-values to zero. Try running your car with f-values of zero. Does your car behave similarly? Does it go straight?

Why do you think you see the behavior you see?

<a id='part4'></a>
# <span style="color:blue">Part 4: Micboard Verification (highly recommended!!!)</span>
-----

We will be using the micboard next week for the SVD/PCA lab! Verify that your biasing circuits and front-end circuitry work as expected.

### Materials:
- Oscilloscope

### Setup:
1. Your mic board VDD should be connected to the 5V rail, which is powered by the 5V voltage regulator.
2. Connect an oscilloscope cable to the output of the micboard.
4. Display a peak-to-peak measurement for the current channel on the oscilloscope.
    - Press "Meas", select Peak-to-Peak, and press "Add measurement"
5. Display an average measurement for the current channel on the oscilloscope.
    - Press "Meas", select Avg - FS, and press "Add measurement"

### Tuning Overview
1. Lower the gain: Turn the potentiometer on your mic board **clockwise**.
2. Increase the gain: Turn the gain potentiometer on the mic board **counter-clockwise** until you see a signal with $\approx2.5\mathrm{V}_{pp}$ amplitude centered near $1.65\mathrm{V}$ while you speak from a comfortable distance away from the microphone.

3. Now place your oscillocope cable at the output of the non-inverting amplifier that follows your low pass filter (`AMP_OUT`). Verify that the output of the amplifier in this circuit is working as expected when you speak.

<img style="width:750px" src="images/sp21_low_pass.png">

Congratulations! You have successfully verified your micboard circuitry.  

<img width='30px' align='left' src="http://inst.eecs.berkeley.edu/~ee16b/sp16/lab_pics/check.png">

## <span style="color:green">CHECKOFF</span>
- <span style="color:green">**Show your GSI that your car moves in a straight line. The definition of straight can be found on the lab's slides!** Your GSI will also check your f-values.</span>
- **Make sure you have read the lab note before requesting a checkoff! Many checkoff questions are pulled straight from the lab note.**

## <span style="color:red">SAVE ALL YOUR DATA!!</span>
- **Data stored on the lab computers often gets deleted automatically.** Please store it on your personal flash drive or cloud storage like Google Drive, and not on the lab computers! If you used DataHub, it should save through your CalNet ID.
- **You will need everything for the final lab report. More details will be announced later in the semester.**